# Collect

In [1]:
import concurrent.futures
import pandas
import requests
from tqdm.notebook import tqdm as tqdm_notebook

## Pokemon

In [2]:
BASE_URL = "https://pokeapi.co/api/v2/pokemon"

count = 0

response = requests.get(BASE_URL)
if response.status_code == 200:
    data = response.json()
    count = data.get("count", 0)

In [3]:
response = requests.get(f"{BASE_URL}?limit={count}")

generations = {
    "generation-i": 1,
    "generation-ii": 2,
    "generation-iii": 3,
    "generation-iv": 4,
    "generation-v": 5,
    "generation-vi": 6,
    "generation-vii": 7,
    "generation-viii": 8,
    "generation-ix": 9,
}

def get_species(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            is_legendary = data.get("is_legendary", False)
            generation_key = data.get("generation", {}).get("name", "")
            return {
                "is_legendary": is_legendary,
                "generation": generations.get(generation_key, "")
            }
        else:
            return {
                "is_legendary": False,
                "generation": ""
            }
    except:
        return {
            "is_legendary": False,
            "generation": ""
        }

def get_pokemon(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            pokemon_id = data.get("id")
            name = data.get("name")
            # Image
            image_url = data.get("sprites", {}).get("front_default", "")
            response = requests.get(image_url)
            if response.status_code == 200:
                image_data = response.content
                image_file = open(f"./images/pokemon/{pokemon_id}.png", 'wb')
                image_file.write(image_data)
            # Species
            species_url = data.get("species", {}).get("url", "")
            species = get_species(species_url)
            # Types
            types = data.get("types", [])
            type_1 = next((item for item in types if item["slot"] == 1), {}).get("type", {}).get("name", "")
            type_2 = next((item for item in types if item["slot"] == 2), {}).get("type", {}).get("name", "")
            # Stats
            stats = data.get("stats", [])
            hp = next(item for item in stats if item.get("stat", {}).get("name", "") == "hp").get("base_stat", 0)
            attack = next(item for item in stats if item.get("stat", {}).get("name", "") == "attack").get("base_stat", 0)
            defense = next(item for item in stats if item.get("stat", {}).get("name", "") == "defense").get("base_stat", 0)
            special_attack = next(item for item in stats if item.get("stat", {}).get("name", "") == "special-attack").get("base_stat", 0)
            special_defense = next(item for item in stats if item.get("stat", {}).get("name", "") == "special-defense").get("base_stat", 0)
            speed = next(item for item in stats if item.get("stat", {}).get("name", "") == "speed").get("base_stat", 0)
            total_base_stat = hp + attack + defense + special_attack + special_defense + speed
            return { 
                "id": pokemon_id,
                "name": name,
                "type_1": type_1,
                "type_2": type_2,
                "hp": hp,
                "attack": attack, 
                "defense": defense, 
                "special_attack": special_attack, 
                "special_defense": special_defense, 
                "speed": speed,
                "total_base_stat": total_base_stat,
                "is_legendary": species.get("is_legendary", False),
                "generation": species.get("generation", "")
            }
        else:
            return {
                "id": 0,
                "name": "",
                "type_1": "",
                "type_2": "",
                "hp": 0,
                "attack": 0, 
                "defense": 0, 
                "special_attack": 0, 
                "special_defense": 0, 
                "speed": 0,
                "total_base_stat": 0,
                "is_legendary": False,
                "generation": ""
            }
    except:
        return {
            "id": 0,
            "name": "",
            "type_1": "",
            "type_2": "",
            "hp": 0,
            "attack": 0, 
            "defense": 0, 
            "special_attack": 0, 
            "special_defense": 0, 
            "speed": 0,
            "total_base_stat": 0,
            "is_legendary": False,
            "generation": ""
        }

list_of_pokemon = []

if response.status_code == 200:
    data = response.json()
    results = data.get("results", [])
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for result in tqdm_notebook(results):
            url = result.get("url")
            futures.append(executor.submit(get_pokemon, url=url))
        for future in tqdm_notebook(concurrent.futures.as_completed(futures)):
            pokemon = future.result()
            list_of_pokemon.append(pokemon)
    pokemon_data_frame = pandas.DataFrame.from_dict(list_of_pokemon)
    sorted_pokemon_data_frame = pokemon_data_frame.sort_values(by = ["id"], ascending = True)
    sorted_pokemon_data_frame.to_csv('data/pokemon.csv', index = False, header = True)

  0%|          | 0/1279 [00:00<?, ?it/s]

0it [00:00, ?it/s]